Alvaro Pequeño Mondragón 1726520 | Datos Masivos - Maestría en Ciencias de Datos

In [1]:
import os
import sys

import findspark
findspark.init()
findspark.find()

import pyspark

import pandas as pd

from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F

In [2]:
spark= SparkSession.builder.appName("Mi primera").getOrCreate()

In [3]:
spark

Se decidió usar una base de datos que contiene los eventos ocurridos en partidos de futbol de partidos de las ligas de España, Inglaterra, Francia, Italia y Alemania. Estos partidos son del año 2011 al 2017, lo que nos da un total de poco mas de 900,000 registros por lo que creo que es una buena base de datos a usar para Datos Masivos.

In [4]:
#Se muestra el diccionario de las diferentes variables para saber que significa cada una
diccionario = open("C:/Users/alvar/Documents/Maestria en Ciencia de datos/Datos Masivos/Soccer Events DB/dictionary.txt", "r")
content = diccionario.read()
print(content)
diccionario.close()

event_type
0	Announcement
1	Attempt
2	Corner
3	Foul
4	Yellow card
5	Second yellow card
6	Red card
7	Substitution
8	Free kick won
9	Offside
10	Hand ball
11	Penalty conceded


event_type2
12	Key Pass
13	Failed through ball
14	Sending off
15	Own goal


side
1	Home
2	Away


shot_place
1	Bit too high
2	Blocked
3	Bottom left corner
4	Bottom right corner
5	Centre of the goal
6	High and wide
7	Hits the bar
8	Misses to the left
9	Misses to the right
10	Too high
11	Top centre of the goal
12	Top left corner
13	Top right corner


shot_outcome
1	On target
2	Off target
3	Blocked
4	Hit the bar


location
1	Attacking half
2	Defensive half
3	Centre of the box
4	Left wing
5	Right wing
6	Difficult angle and long range
7	Difficult angle on the left
8	Difficult angle on the right
9	Left side of the box
10	Left side of the six yard box
11	Right side of the box
12	Right side of the six yard box
13	Very close range
14	Penalty spot
15	Outside the box
16	Long range
17	More than 35 yards
18	More than 40 yards
19

In [5]:
#Se lee la base de datos a través de spark
df = spark.read.csv('C:/Users/alvar/Documents/Maestria en Ciencia de datos/Datos Masivos/Soccer Events DB/events.csv', header=True, inferSchema=True)

In [6]:
df.printSchema()

root
 |-- id_odsp: string (nullable = true)
 |-- id_event: string (nullable = true)
 |-- sort_order: integer (nullable = true)
 |-- time: integer (nullable = true)
 |-- text: string (nullable = true)
 |-- event_type: integer (nullable = true)
 |-- event_type2: string (nullable = true)
 |-- side: integer (nullable = true)
 |-- event_team: string (nullable = true)
 |-- opponent: string (nullable = true)
 |-- player: string (nullable = true)
 |-- player2: string (nullable = true)
 |-- player_in: string (nullable = true)
 |-- player_out: string (nullable = true)
 |-- shot_place: string (nullable = true)
 |-- shot_outcome: string (nullable = true)
 |-- is_goal: integer (nullable = true)
 |-- location: string (nullable = true)
 |-- bodypart: string (nullable = true)
 |-- assist_method: integer (nullable = true)
 |-- situation: string (nullable = true)
 |-- fast_break: integer (nullable = true)



In [7]:
#Se ven estadisticas descriptivas básicas de los datos
df.describe().show()

+-------+---------+----------+------------------+------------------+--------------------+-----------------+------------------+------------------+----------+----------+------+-------+---------+--------------+-----------------+------------------+-------------------+-----------------+------------------+-------------------+------------------+--------------------+
|summary|  id_odsp|  id_event|        sort_order|              time|                text|       event_type|       event_type2|              side|event_team|  opponent|player|player2|player_in|    player_out|       shot_place|      shot_outcome|            is_goal|         location|          bodypart|      assist_method|         situation|          fast_break|
+-------+---------+----------+------------------+------------------+--------------------+-----------------+------------------+------------------+----------+----------+------+-------+---------+--------------+-----------------+------------------+-------------------+------------

Como primer ejercicio se decidio tratar de ver que jugadores que entran de cambio tienen un mayor cantidad de goles anotados para de esta manera encontrar aquellos jugadores que hacen una diferencia al entrar de cambio.

In [8]:
#Primero se filtro aquellos eventos que corresponden a un cambio de jugador y que se hayan hecho despues del minuto 85 y tambien aquellos eventos que corresponden a un gol y que fueron anotados despues del minuto 80.
#De esta manera nos quedamos con aquellos jugadores que regularmente entran de cambio y se dejan las columnas de player (jugadores que anotan gol) y la de player_in(jugadores que entraron de cambio).
sub_goal = df.filter(((df.event_type == 7) & (df.time > 85)) | ((df.is_goal == 1) & (df.time > 80))).select('event_type', 'player', 'player_in', 'is_goal')
sub_goal.show()

+----------+----------------+-----------------+-------+
|event_type|          player|        player_in|is_goal|
+----------+----------------+-----------------+-------+
|         7|              NA|   sebastian kehl|      0|
|         1|  sascha molders|               NA|      1|
|         7|              NA|    garra dembele|      0|
|         1|markus rosenberg|               NA|      1|
|         7|              NA|    daniel didavi|      0|
|         7|              NA|guillaume rippert|      0|
|         1|  marcel schafer|               NA|      1|
|         7|              NA|       andrezinho|      0|
|         1|  patrick helmes|               NA|      1|
|         7|              NA|    ronny rodelin|      0|
|         7|              NA|      jogook jung|      0|
|         7|              NA|      rudy haddad|      0|
|         1|  olivier giroud|               NA|      1|
|         1|  shinji okazaki|               NA|      1|
|         7|              NA|     prince tagoe| 

In [9]:
#Se guarda en una tabla a parte los nombres de los jugadores que entraron de cambio y se eliminó de la lista aquellos registros con NA
sub_players = sub_goal.select('player_in').distinct().filter(sub_goal.player_in != 'NA')
sub_players.show()

+-------------------+
|          player_in|
+-------------------+
|             joselu|
|     sergio sanchez|
|       pablo piatti|
|      mevlut erdinc|
|    jonathan brison|
|        rui sampaio|
|       djamel bakar|
|      jan kirchhoff|
|      alberto botia|
| jean philippe sabo|
|   cyril jeunechamp|
|      amadou sidibe|
|christoph metzelder|
|        jan moravek|
|       daniel guiza|
|     carlos carmona|
|     javier camunas|
|        gaetan bong|
|       gianni bruno|
|fabian lustenberger|
+-------------------+
only showing top 20 rows



In [10]:
#Se agrupo la columna de player (Jugadores que han metido gol) y se creo una columna con la suma de los goles anotados.
sub_goal = sub_goal.groupby('player').sum('is_goal')
sub_goal.show()

+--------------------+------------+
|              player|sum(is_goal)|
+--------------------+------------+
|        henri bedimo|           2|
|       mevlut erdinc|          14|
|   franck dja djedje|           1|
|        brice jovial|           1|
|              paulao|           2|
|            bernardo|           1|
|        lewis holtby|           7|
|      eduardo salvio|           1|
|         oscar trejo|           4|
|      bosko jankovic|           2|
|      daniel niculae|           2|
|      benjamin andre|           2|
|      edinson cavani|          20|
|          dani alves|           1|
|        jorge molina|           5|
| ruud van nistelrooy|           2|
|         david lopez|           3|
|pierremichel lasogga|           6|
|          moussa sow|           1|
|          arda turan|           4|
+--------------------+------------+
only showing top 20 rows



In [11]:
#por ultimo se hace un inner join de la tabla de jugadores que han anotado gol con la tabla de jugadores que entran de cambio para quedarnos solo con aquellos jugadores que han entrado de cambio.
#Despues del join se ordenan los datos por numero de goles de mayor a menor y podemos visualizar aquellos jugadores mas determinantes que entran de cambio en los ultimos minutos del juego.
sub_goal.join(sub_players, sub_goal.player == sub_players.player_in, 'inner').select('player', 'sum(is_goal)').orderBy(F.desc("sum(is_goal)")).show(100)

+--------------------+------------+
|              player|sum(is_goal)|
+--------------------+------------+
| alexandre lacazette|          18|
|     gonzalo higuain|          16|
|       jose callejon|          16|
|        ruben castro|          15|
|         luis suarez|          15|
|       mevlut erdinc|          14|
|     bafetimbi gomis|          14|
|       alvaro morata|          14|
|  andrepierre gignac|          13|
|    mauricio pinilla|          13|
|     mario balotelli|          13|
|    mame biram diouf|          13|
|       nils petersen|          11|
|        mauro icardi|          11|
|       kevin gameiro|          11|
|      miroslav klose|          11|
|     roberto soldado|          11|
|    stefan kiessling|          11|
|     anthony modeste|          11|
|                eder|          11|
|     rodrigo palacio|          11|
|              nolito|          11|
|        ivan perisic|          10|
|      alvaro negredo|          10|
|      cedric bakambu|      

Para el segundo ejemplo obtendremos la suma de las tarjetas recibidas por jugador para poder visualizar que jugadores son los mas indisciplinados

In [68]:
#Obtenemos la cantidad de tarjetas amarillas por jugador
tarjetas_am = df.filter(df.event_type == 4).select('player', 'event_type').groupby('player').agg(F.count(df.event_type).alias("amarillas")).orderBy(F.desc("amarillas")).withColumnRenamed("player", "player1")
tarjetas_am.show()

+-----------------+---------+
|          player1|amarillas|
+-----------------+---------+
|         juanfran|       78|
|   victor sanchez|       72|
|             gabi|       65|
|  alvaro gonzalez|       61|
|           danilo|       60|
|      ruben perez|       60|
|    bostjan cesar|       59|
|     sergio ramos|       59|
|alejandro arribas|       58|
|           alexis|       58|
|   vicente iborra|       58|
|    luca cigarini|       57|
|        pape diop|       57|
| radja nainggolan|       56|
|      luca rigoni|       56|
|       gary medel|       56|
|    daniele conti|       56|
|       javi fuego|       53|
|            nacho|       53|
|  ignacio camacho|       53|
+-----------------+---------+
only showing top 20 rows



In [69]:
#Obtenemos la cantidad de segundas tarjetas amarillas por jugador
tarjetas_2am = df.filter(df.event_type == 5).select('player', 'event_type').groupby('player').agg(F.count(df.event_type).alias("seg_amarillas")).orderBy(F.desc("seg_amarillas")).withColumnRenamed("player", "player2")
tarjetas_2am.show()

+-------------------+-------------+
|            player2|seg_amarillas|
+-------------------+-------------+
|        sergi gomez|            2|
|   alessandro gazzi|            2|
|    yannick cahuzac|            2|
|jeanphilippe gbamin|            2|
|      james tomkins|            1|
|       franck beria|            1|
|     carlos carmona|            1|
|    aaron cresswell|            1|
|        harry arter|            1|
|        pedro bigas|            1|
|         javi fuego|            1|
|        leon guwara|            1|
|  slobodan rajkovic|            1|
|      darko lazovic|            1|
|   zinedine machach|            1|
|         ruben vezo|            1|
|             zaldua|            1|
|        ever banega|            1|
|      enzo crivelli|            1|
|          edenilson|            1|
+-------------------+-------------+
only showing top 20 rows



In [70]:
#Obtenemos la cantidad de tarjetas rojas por jugador
tarjetas_ro = df.filter(df.event_type == 6).select('player', 'event_type').groupby('player').agg(F.count(df.event_type).alias("rojas")).orderBy(F.desc("rojas")).withColumnRenamed("player", "player3")
tarjetas_ro.show()

+-------------------+-----+
|            player3|rojas|
+-------------------+-----+
|    younes belhanda|    4|
|   thibaut courtois|    4|
|    mouhamadou dabo|    4|
|      fallou diagne|    4|
|             hilton|    4|
|daniele gastaldello|    4|
|      daniel congre|    4|
|          wes brown|    4|
|             michel|    3|
|               tono|    3|
|        uros spajic|    3|
|     vedad ibisevic|    3|
|kevinprince boateng|    3|
|     franck tabanou|    3|
|     federico fazio|    3|
|   mauricio pinilla|    3|
|      abdel barrada|    3|
|       goran pandev|    3|
|          rod fanni|    3|
|        diego lopez|    3|
+-------------------+-----+
only showing top 20 rows



In [71]:
#Se realiza un join para tener por columnas separadas el tipo de tarjetas por jugador
tarjetas = tarjetas_am.join(tarjetas_2am, tarjetas_am.player1 ==  tarjetas_2am.player2, 'outer')
tarjetas = tarjetas.join(tarjetas_ro, tarjetas.player1 ==  tarjetas_ro.player3, 'outer').select('player1', 'amarillas', 'seg_amarillas', 'rojas').filter(tarjetas.player1 != 'NULL')
tarjetas.show()

+------------------+---------+-------------+-----+
|           player1|amarillas|seg_amarillas|rojas|
+------------------+---------+-------------+-----+
|       bo svensson|        9|         NULL| NULL|
|      henri bedimo|       19|         NULL| NULL|
|      pablo piatti|       17|         NULL| NULL|
|             ruben|       10|         NULL| NULL|
|    sergio sanchez|       29|         NULL|    1|
|  knowledge musona|        4|         NULL| NULL|
|    adam matuschyk|        1|         NULL| NULL|
|    daniele bonera|       29|         NULL| NULL|
|            joselu|       11|         NULL| NULL|
|   jonathan brison|       18|         NULL|    1|
|     mevlut erdinc|        5|         NULL|    1|
|alhassane bangoura|        1|         NULL| NULL|
|        yann mvila|        5|         NULL| NULL|
|           carmelo|        6|         NULL| NULL|
|            danilo|       60|         NULL|    1|
|     victor valdes|        7|         NULL| NULL|
|      kiko casilla|       10| 

In [73]:
#Pasamos los valores nulos a 0
tarjetas =  tarjetas.na.fill(0)
tarjetas.show()

+------------------+---------+-------------+-----+
|           player1|amarillas|seg_amarillas|rojas|
+------------------+---------+-------------+-----+
|       bo svensson|        9|            0|    0|
|      henri bedimo|       19|            0|    0|
|      pablo piatti|       17|            0|    0|
|             ruben|       10|            0|    0|
|    sergio sanchez|       29|            0|    1|
|  knowledge musona|        4|            0|    0|
|    adam matuschyk|        1|            0|    0|
|    daniele bonera|       29|            0|    0|
|            joselu|       11|            0|    0|
|   jonathan brison|       18|            0|    1|
|     mevlut erdinc|        5|            0|    1|
|alhassane bangoura|        1|            0|    0|
|        yann mvila|        5|            0|    0|
|           carmelo|        6|            0|    0|
|            danilo|       60|            0|    1|
|     victor valdes|        7|            0|    0|
|      kiko casilla|       10| 

In [85]:
#Se suman todas las tarjetas recibidas por jugador y se ordenan de mayor a menor 
trajetas = tarjetas.groupby('player1').agg(F.sum(tarjetas.amarillas + tarjetas.seg_amarillas + tarjetas.rojas).alias('tarjetas_totales')).orderBy(F.desc("tarjetas_totales"))
trajetas.show()

+-----------------+----------------+
|          player1|tarjetas_totales|
+-----------------+----------------+
|         juanfran|              82|
|   victor sanchez|              73|
|             gabi|              65|
|  alvaro gonzalez|              63|
|           danilo|              61|
|           alexis|              61|
|      ruben perez|              60|
|     sergio ramos|              60|
|    bostjan cesar|              59|
|alejandro arribas|              59|
|        pape diop|              59|
|    luca cigarini|              58|
|       gary medel|              58|
|   vicente iborra|              58|
|    daniele conti|              57|
|      luca rigoni|              56|
| radja nainggolan|              56|
|   carlos carmona|              54|
|       javi fuego|              54|
|      raul garcia|              53|
+-----------------+----------------+
only showing top 20 rows

